


FASTA
=====

--------

This notebook is a simplified version of the one originally developed by [Ben Langmead](https://github.com/BenLangmead/ads1-notebooks).

-----

This notebook briefly explores the [FASTA] format, a very common format for storing DNA sequences.  FASTA is the preferred format for storing *reference genomes*.

FASTA and FASTQ are rather similar, but FASTQ is almost always used for storing *sequencing reads* (with associated quality values), whereas FASTA is used for storing all kinds of DNA, RNA or protein sequencines (without associated quality values).

Before delving into the format, I should mention that the [BioPython] project makes parsing and using many file formats, including FASTA, quite easy.  See the BioPython [SeqIO] module in particular.  As far as I know, though, [SeqIO] does not use FASTA indexes, discussed toward the bottom, which is a disadvantage.

[FASTA]: http://en.wikipedia.org/wiki/FASTA_format
[BioPython]: http://biopython.org/wiki/Main_Page
[SeqIO]: http://biopython.org/wiki/SeqIO

### Basic format

Here is the basic format:

    >sequence1_short_name with optional additional info after whitespace
    ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
    GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGC
    AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
    >sequence2_short_name with optional additional info after whitespace
    GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
    ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
    ATATAG

A line starting with a `>` (greater-than) sign indicates the beginning of a new sequence and specifies its name.  Take the first line above.  Everything after the `>` up to and excluding the first whitespace character (`sequence1_short_name`), is the "short name."  Everything after the `>` up to the end of the *line* (`sequence1_short_name with optional additional info after whitespace`) is the "long name."  We usually use the short name when referring to FASTA sequences.

The next three lines consists of several nucleotides.  There is a maximum number of nucleotides permitted per line; in this case, it is 70.  If the sequence is longer then 70 nucleotides, it "wraps" down to the next line.  Not every FASTA file uses the same maximum, but a given FASTA file must use the same maximum throughout the file.

The sequences above are made up.  Here's a real-world reference sequence of SARS-CoV-2 in FASTA format:


In [4]:
import gzip
import urllib.request
url = 'ftp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/chromosomes/NC_045512v2.fa.gz'
response = urllib.request.urlopen(url)
print(gzip.decompress(response.read()).decode('UTF8'))

>NC_045512v2
...
AAA



In [5]:
!wget ftp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/chromosomes/NC_045512v2.fa.gz

--2021-01-21 15:13:37--  ftp://hgdownload.soe.ucsc.edu/goldenPath/wuhCor1/chromosomes/NC_045512v2.fa.gz
           => ‘NC_045512v2.fa.gz’
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /goldenPath/wuhCor1/chromosomes ... done.
==> SIZE NC_045512v2.fa.gz ... 9632
==> PASV ... done.    ==> RETR NC_045512v2.fa.gz ... done.
Length: 9632 (9.4K) (unauthoritative)

NC_045512v2.fa.gz   100%[===================>]   9.41K  --.-KB/s    in 0s      

2021-01-21 15:13:39 (24.9 MB/s) - ‘NC_045512v2.fa.gz’ saved [9632]



In [6]:
!ls

NC_045512v2.fa.gz  sample_data


In [7]:
!gunzip NC_045512v2.fa.gz

This FASTA file shown above has just one sequence in it.  As we saw in the first example above, it's also possible for one FASTA file to contain multiple sequences.  These are sometimes called multi-FASTA files.  When you write code to interpret FASTA files, it's a good idea to *always* allow for the possibility that the FASTA file might contain multiple sequences.

FASTA files are often stored with the `.fa` file name extension, but this is not a rule.  `.fasta` is another popular extenson.  You may also see `.fas`, `.fna`, `.mfa` (for multi-FASTA), and others.

### Parsing FASTA

Here is a simple function for parsing a FASTA file into a Python dictionary.  The dictionary maps short names to corresponding nucleotide strings (with whitespace removed).

In [8]:
def parse_fasta(fh):
    fa = {}
    current_short_name = None
    # Part 1: compile list of lines per sequence
    for ln in fh:
        if ln[0] == '>':
            # new name line; remember current sequence's short name
            long_name = ln[1:].rstrip()
            current_short_name = long_name.split()[0]
            fa[current_short_name] = []
        else:
            # append nucleotides to current sequence
            fa[current_short_name].append(ln.rstrip())
    # Part 2: join lists into strings
    for short_name, nuc_list in fa.items():
        # join this sequence's lines into one long string
        fa[short_name] = ''.join(nuc_list)
    return fa

The first part accumulates a list of strings (one per line) for each sequence.  The second part joins those lines together so that we end up with one long string per sequence.  Why divide it up this way?  Mainly to avoid the [poor performance](http://www.skymind.com/~ocrow/python_string/) of repeatedly concatenating (immutable) Python strings.

I'll test it by running it on the simple multi-FASTA file we saw before:

In [3]:
from io import StringIO
fasta_example = StringIO(
'''>sequence1_short_name with optional additional info after whitespace
ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAA
GCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGC
AATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT
>sequence2_short_name with optional additional info after whitespace
GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCG
ATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATA
ATATAG''')
parsed_fa = parse_fasta(fasta_example)
parsed_fa

{'sequence1_short_name': 'ACATCACCCCATAAACAAATAGGTTTGGTCCTAGCCTTTCTATTAGCTCTTAGTAAGATTACACATGCAAGCATCCCCGTTCCAGTGAGTTCACCCTCTAAATCACCACGATCAAAAGGAACAAGCATCAAGCACGCAGCAATGCAGCTCAAAACGCTTAGCCTAGCCACACCCCCACGGGAAACAGCAGTGAT',
 'sequence2_short_name': 'GCCCCAAACCCACTCCACCTTACTACCAGACAACCTTAGCCAAACCATTTACCCAAATAAAGTATAGGCGATAGAAATTGAAACCTGGCGCAATAGATATAGTACCGCAAGGGAAAGATGAAAAATTATAACCAAGCATAATATAG'}

Note that only the short names survive.  This is usually fine, but it's not hard to modify the function so that information relating short names to long names is also retained.

### Other resources

* [Wikipedia page for FASTA format](http://en.wikipedia.org/wiki/Fasta_format)
* The [original FASTA paper](http://www.sciencedirect.com/science/article/pii/007668799083007V) by Bill Pearson.  This is the software tool that made the format popular.
* [BioPython], which has [its own ways of parsing FASTA](http://biopython.org/wiki/SeqIO)
* [Many](https://github.com/brentp/pyfasta) [other](https://github.com/lh3/seqtk) [libraries](https://github.com/mdshw5/pyfaidx) and [tools]((http://hannonlab.cshl.edu/fastx_toolkit/)

[BioPython]: http://biopython.org/wiki/Main_Page
[SeqIO]: http://biopython.org/wiki/SeqIO